정답, 베이스라인, 방법론 적용한 모델의 출력을 비교하기 위해 만든 Jupyter Notebook

- answer_file: 정답 validation 파일
- base_prediction: 아무것도 적용 안 한 베이스 모델의 예측 json 경로
- prediction_file: 특정 방법론을 적용한 모델의 예측 json 경로

예시 출력:

## 모델 성능 비교

| 모델       | 맞춘 갯수 | 틀린 갯수 | 정확도 (%) |
|------------|-----------|-----------|------------|
| 베이스라인 | 161 | 79 | 67.08 |
| 예측 모델  | 158 | 82 | 65.83 |

## 분석 통계

| 항목 | 수량 | 비율 (%) |
| --- | --- | --- |
| 전체 예제 수 | 240 | 100.00 |
| 같은 예측 수 | 162 | 67.50 |
| 다른 예측 수 | 78 | 32.50 |
| - 베이스 모델이 맞춘 경우 | 33 | 13.75 |
| - 어쩌구 저쩌구 모델이 맞춘 경우 | 19 | 7.92 |
| - 둘 다 틀린 경우 | 26 | 10.83 |

## 같은 예측 항목들

| ID | 질문 | 정답 | 예측 | 정답 여부 |
| --- | --- | --- | --- | --- |
| mrc-0-003264 | 처음으로 부실 경영인에 대한 보상 선고를 받은 회사는? | 한보철강 | 한보철강 | ✅ |
| mrc-1-001810 | 촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은? | 나뭇잎 | 나뭇잎 | ✅ |
| mrc-1-000219 | 로타이르가 백조를 구하기 위해 사용한 것은? | 금대야 | 금대야 | ✅ |

## 베이스 모델이 맞추고 어쩌구 저쩌구 모델이 틀린 항목들

| ID | 질문 | 정답 | 베이스 예측 | 어쩌구 저쩌구 모델 예측 |
| --- | --- | --- | --- | --- |
| mrc-0-004762 | 스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는? | 1871년 | 1871년 | 1868년 |
| mrc-0-005106 | 1945년 쇼와천황의 항복 선언이 발표된 라디오 방송은? | 옥음방송 | 옥음방송 | 쇼와 천황의 옥음방송 |

## 어쩌구 저쩌구 모델이 맞추고 베이스 모델이 틀린 항목들

| ID | 질문 | 정답 | 어쩌구 저쩌구 모델 예측 | 베이스 예측 |
| --- | --- | --- | --- | --- |
| mrc-1-000414 | 다른 과 의사들은 감염내과 전문의들로부터 어떤 것에 대해 조언을 받는가? | 복잡한 감염병 | 복잡한 감염병 | 감염병 환자를 진단하거나 관리하기 어려운 경우 |
| mrc-0-003032 | 반대동맹이 공산당과 갈라서겠다고 얘기한 날은 언제인가? | 1967년 11월 15일 | 1967년 11월 15일 | 1967년 8월 16일 |

## 둘 다 틀린 항목들

| ID | 질문 | 정답 | 어쩌구 저쩌구 모델 예측 | 베이스 예측 |
| --- | --- | --- | --- | --- |
| mrc-0-002138 | 몽케가 죽은 뒤 쿠릴타이에서 대칸의 지위를 얻은 사람의 이름은? | 아리크 부케 | 쿠빌라이 | 막냇동생 아리크 부케 |
| mrc-0-001022 | 칭자오의 머리가 엄청난 위력을 발휘할 수 없게 된 것은 누구 때문인가? | 몽키 D. 가프 | 할아버지 | 루피의 할아버지 |



In [1]:
import json
from datasets import load_dataset

# 정답 파일 경로
answer_file = "/data/ephemeral/home/sangyeop/level2-mrc-nlp-11/data/default/validation/data-00000-of-00001.arrow"
base_prediction="/data/ephemeral/home/sangyeop/level2-mrc-nlp-11/outputs/bz=16_lr=1.7110631470130408e-05_eval_predictions.json"
# 예측 파일 경로
prediction_file = "/data/ephemeral/home/sangyeop/level2-mrc-nlp-11/outputs/lora_True_title_context_merge_token_default_bz=16_lr=1.6764783497920226e-05_eval_predictions.json"
model_name="어쩌구 저쩌구"
# 정답 데이터 로드
# Arrow 파일은 Hugging Face Datasets 라이브러리를 사용하여 로드할 수 있습니다.
dataset = load_dataset('arrow', data_files=answer_file, split='train')  # split 이름은 실제에 맞게 조정
    
# 예측 데이터 로드
with open(prediction_file, 'r', encoding='utf-8') as f:
    predictions = json.load(f)
with open(base_prediction, 'r', encoding='utf-8') as f:
    base_prediction = json.load(f)


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
baseline_correct = 0
baseline_incorrect = 0
prediction_correct = 0
prediction_incorrect = 0


# 각 예제에 대해 두 모델의 예측을 정답과 비교
for example in dataset:
    example_id = str(example['id'])  # ID를 문자열로 변환하여 일치시킴
    true_answers = example['answers']['text']
    true_answer = true_answers[0] if true_answers else ""  # 첫 번째 정답 사용 (존재하지 않으면 빈 문자열)

    # 베이스라인 모델의 예측
    base_pred = base_prediction.get(example_id, "")
    if base_pred == true_answer:
        baseline_correct += 1
    else:
        baseline_incorrect += 1

    # 예측 모델의 예측
    pred = predictions.get(example_id, "")
    if pred == true_answer:
        prediction_correct += 1
    else:
        prediction_incorrect += 1

# 전체 예제 수
total_examples = len(dataset)

# 정확도 계산
baseline_accuracy = (baseline_correct / total_examples) * 100
prediction_accuracy = (prediction_correct / total_examples) * 100

# 마크다운 표 출력
markdown_table = f"""
## 모델 성능 비교

| 모델       | 맞춘 갯수 | 틀린 갯수 | 정확도 (%) |
|------------|-----------|-----------|------------|
| 베이스라인 | {baseline_correct} | {baseline_incorrect} | {baseline_accuracy:.2f} |
| 예측 모델  | {prediction_correct} | {prediction_incorrect} | {prediction_accuracy:.2f} |
"""

print(markdown_table)



## 모델 성능 비교

| 모델       | 맞춘 갯수 | 틀린 갯수 | 정확도 (%) |
|------------|-----------|-----------|------------|
| 베이스라인 | 161 | 79 | 67.08 |
| 예측 모델  | 166 | 74 | 69.17 |



In [3]:
# 예측 비교 결과를 저장할 리스트 초기화
same_predictions = []
different_predictions = []

# 추가 분류를 위한 리스트 초기화
base_correct_predictions = []
predicted_correct_predictions = []
both_incorrect_predictions = []

# 각 예제에 대해 두 예측을 비교
for example in dataset:
    example_id = example['id']  # 예제의 ID
    true_answers = example['answers']['text']  # 정답 리스트
    true_answer = true_answers[0] if true_answers else ""  # 첫 번째 정답 사용 (존재하지 않으면 빈 문자열)

    predicted_answer = predictions.get(str(example_id), "")  # {model_name} 모델의 예측 (ID를 문자열로 변환)
    base_predicted_answer = base_prediction.get(str(example_id), "")  # 베이스 모델의 예측 (ID를 문자열로 변환)

    if predicted_answer == base_predicted_answer:
        same_predictions.append({
            'ID': example_id,
            '질문': example['question'],
            '컨텍스트': example['context'],
            '정답': true_answer,
            '예측': predicted_answer
        })
    else:
        # 두 모델의 예측이 다른 경우 추가 분류
        is_base_correct = base_predicted_answer == true_answer
        is_predicted_correct = predicted_answer == true_answer

        if is_base_correct and not is_predicted_correct:
            base_correct_predictions.append({
                'ID': example_id,
                '질문': example['question'],
                '컨텍스트': example['context'],
                '정답': true_answer,
                '베이스 예측': base_predicted_answer,
                f'{model_name} 모델 예측': predicted_answer
            })
        elif is_predicted_correct and not is_base_correct:
            predicted_correct_predictions.append({
                'ID': example_id,
                '질문': example['question'],
                '컨텍스트': example['context'],
                '정답': true_answer,
                f'{model_name} 모델 예측': predicted_answer,
                '베이스 예측': base_predicted_answer
            })
        else:
            both_incorrect_predictions.append({
                'ID': example_id,
                '질문': example['question'],
                '컨텍스트': example['context'],
                '정답': true_answer,
                f'{model_name} 모델 예측': predicted_answer,
                '베이스 예측': base_predicted_answer
            })

# 전체 예제 수와 통계 계산
total_examples = len(dataset)
same_count = len(same_predictions)
base_correct_count = len(base_correct_predictions)
predicted_correct_count = len(predicted_correct_predictions)
both_incorrect_count = len(both_incorrect_predictions)

different_count = base_correct_count + predicted_correct_count + both_incorrect_count

same_ratio = (same_count / total_examples) * 100
different_ratio = (different_count / total_examples) * 100
base_correct_ratio = (base_correct_count / total_examples) * 100
predicted_correct_ratio = (predicted_correct_count / total_examples) * 100
both_incorrect_ratio = (both_incorrect_count / total_examples) * 100

# 통계 출력 (마크다운 표 형식)
print("## 분석 통계")
print()
print("| 항목 | 수량 | 비율 (%) |")
print("| --- | --- | --- |")
print(f"| 전체 예제 수 | {total_examples} | 100.00 |")
print(f"| 같은 예측 수 | {same_count} | {same_ratio:.2f} |")
print(f"| 다른 예측 수 | {different_count} | {different_ratio:.2f} |")
print(f"| - 베이스 모델이 맞춘 경우 | {base_correct_count} | {base_correct_ratio:.2f} |")
print(f"| - {model_name} 모델이 맞춘 경우 | {predicted_correct_count} | {predicted_correct_ratio:.2f} |")
print(f"| - 둘 다 틀린 경우 | {both_incorrect_count} | {both_incorrect_ratio:.2f} |")
print()

# 같은 예측 항목 출력 (마크다운 표 형식) 및 정답 여부 표시
print("## 같은 예측 항목들")
print()
if same_predictions:
    print("| ID | 질문 | 정답 | 예측 | 정답 여부 |")
    print("| --- | --- | --- | --- | --- |")
    for item in same_predictions:
        # 마크다운 표에서는 파이프(|) 문자를 이스케이프해야 합니다.
        question = item['질문'].replace("|", "\\|")
        true_answer = item['정답'].replace("|", "\\|")
        predicted_answer = item['예측'].replace("|", "\\|")
        
        # 정답 여부 판단
        is_correct = "✅" if item['예측'] == item['정답'] else "❌"
        
        print(f"| {item['ID']} | {question} | {true_answer} | {predicted_answer} | {is_correct} |")
else:
    print("같은 예측 항목이 없습니다.")
print()


# 베이스 모델이 맞추고 {model_name} 모델이 틀린 항목 출력 (마크다운 표 형식)
print(f"## 베이스 모델이 맞추고 {model_name} 모델이 틀린 항목들")
print()
if base_correct_predictions:
    print(f"| ID | 질문 | 정답 | 베이스 예측 | {model_name} 모델 예측 |")
    print("| --- | --- | --- | --- | --- |")
    for item in base_correct_predictions:
        question = item['질문'].replace("|", "\\|")
        true_answer = item['정답'].replace("|", "\\|")
        base_pred = item['베이스 예측'].replace("|", "\\|")
        category_pred = item[f'{model_name} 모델 예측'].replace("|", "\\|")
        print(f"| {item['ID']} | {question} | {true_answer} | {base_pred} | {category_pred} |")
else:
    print(f"베이스 모델이 맞추고 {model_name} 모델이 틀린 항목이 없습니다.")
print()

# {model_name} 모델이 맞추고 베이스 모델이 틀린 항목 출력 (마크다운 표 형식)
print(f"## {model_name} 모델이 맞추고 베이스 모델이 틀린 항목들")
print()
if predicted_correct_predictions:
    print(f"| ID | 질문 | 정답 | {model_name} 모델 예측 | 베이스 예측 |")
    print("| --- | --- | --- | --- | --- |")
    for item in predicted_correct_predictions:
        question = item['질문'].replace("|", "\\|")
        true_answer = item['정답'].replace("|", "\\|")
        category_pred = item[f'{model_name} 모델 예측'].replace("|", "\\|")
        base_pred = item['베이스 예측'].replace("|", "\\|")
        print(f"| {item['ID']} | {question} | {true_answer} | {category_pred} | {base_pred} |")
else:
    print(f"{model_name} 모델이 맞추고 베이스 모델이 틀린 항목이 없습니다.")
print()

# 둘 다 틀린 항목 출력 (마크다운 표 형식)
print("## 둘 다 틀린 항목들")
print()
if both_incorrect_predictions:
    print(f"| ID | 질문 | 정답 | {model_name} 모델 예측 | 베이스 예측 |")
    print("| --- | --- | --- | --- | --- |")
    for item in both_incorrect_predictions:
        question = item['질문'].replace("|", "\\|")
        true_answer = item['정답'].replace("|", "\\|")
        category_pred = item[f'{model_name} 모델 예측'].replace("|", "\\|")
        base_pred = item['베이스 예측'].replace("|", "\\|")
        print(f"| {item['ID']} | {question} | {true_answer} | {category_pred} | {base_pred} |")
else:
    print("둘 다 틀린 항목이 없습니다.")


## 분석 통계

| 항목 | 수량 | 비율 (%) |
| --- | --- | --- |
| 전체 예제 수 | 240 | 100.00 |
| 같은 예측 수 | 178 | 74.17 |
| 다른 예측 수 | 62 | 25.83 |
| - 베이스 모델이 맞춘 경우 | 17 | 7.08 |
| - 어쩌구 저쩌구 모델이 맞춘 경우 | 22 | 9.17 |
| - 둘 다 틀린 경우 | 23 | 9.58 |

## 같은 예측 항목들

| ID | 질문 | 정답 | 예측 | 정답 여부 |
| --- | --- | --- | --- | --- |
| mrc-0-003264 | 처음으로 부실 경영인에 대한 보상 선고를 받은 회사는? | 한보철강 | 한보철강 | ✅ |
| mrc-0-004762 | 스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는? | 1871년 | 1871년 | ✅ |
| mrc-1-001810 | 촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은? | 나뭇잎 | 나뭇잎 | ✅ |
| mrc-1-000219 | 로타이르가 백조를 구하기 위해 사용한 것은? | 금대야 | 금대야 | ✅ |
| mrc-1-000285 | 의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가? | 수평적 관계 | 수평적 관계 | ✅ |
| mrc-0-005106 | 1945년 쇼와천황의 항복 선언이 발표된 라디오 방송은? | 옥음방송 | 옥음방송 | ✅ |
| mrc-0-002076 | 징금수는 서양 자수의 어떤 기법과 같은 기술을 사용하는가? | 코칭 스티치 | 코칭 스티치 | ✅ |
| mrc-0-002875 | 루이 14세의 왕비 마리아 테래사는 어느 나라 공주인가? | 스페인 | 스페인 | ✅ |
| mrc-0-003828 | 헤자즈 왕국이 실존했던 것은 언제까지인가? | 20세기 초 | 20세기 초 | ✅ |
| mrc-0-002778 | 버드 교장이 5월의 여왕의 대안으로 제시한 것은? | "5월의 왕" | "5월의 왕" | ✅ |
| mrc

## 분석 통계

| 항목 | 수량 | 비율 (%) |
| --- | --- | --- |
| 전체 예제 수 | 240 | 100.00 |
| 같은 예측 수 | 178 | 74.17 |
| 다른 예측 수 | 62 | 25.83 |
| - 베이스 모델이 맞춘 경우 | 17 | 7.08 |
| - 어쩌구 저쩌구 모델이 맞춘 경우 | 22 | 9.17 |
| - 둘 다 틀린 경우 | 23 | 9.58 |

## 같은 예측 항목들

| ID | 질문 | 정답 | 예측 | 정답 여부 |
| --- | --- | --- | --- | --- |
| mrc-0-003264 | 처음으로 부실 경영인에 대한 보상 선고를 받은 회사는? | 한보철강 | 한보철강 | ✅ |
| mrc-0-004762 | 스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는? | 1871년 | 1871년 | ✅ |
| mrc-1-001810 | 촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은? | 나뭇잎 | 나뭇잎 | ✅ |
| mrc-1-000219 | 로타이르가 백조를 구하기 위해 사용한 것은? | 금대야 | 금대야 | ✅ |
| mrc-1-000285 | 의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가? | 수평적 관계 | 수평적 관계 | ✅ |
| mrc-0-005106 | 1945년 쇼와천황의 항복 선언이 발표된 라디오 방송은? | 옥음방송 | 옥음방송 | ✅ |
| mrc-0-002076 | 징금수는 서양 자수의 어떤 기법과 같은 기술을 사용하는가? | 코칭 스티치 | 코칭 스티치 | ✅ |
| mrc-0-002875 | 루이 14세의 왕비 마리아 테래사는 어느 나라 공주인가? | 스페인 | 스페인 | ✅ |
| mrc-0-003828 | 헤자즈 왕국이 실존했던 것은 언제까지인가? | 20세기 초 | 20세기 초 | ✅ |
| mrc-0-002778 | 버드 교장이 5월의 여왕의 대안으로 제시한 것은? | "5월의 왕" | "5월의 왕" | ✅ |
| mrc-0-003931 | 인형사'를 만들어낸 것으로 추측되는 사업의 이름은? | '일급 비밀 프로젝트 2501' | '일급 비밀 프로젝트 2501' | ✅ |
| mrc-0-002485 | 멘데스가 요원들을 구하기 위해 간 도시는 어디인가? | 테헤란 | 테헤란 | ✅ |
| mrc-0-004483 | 교과부의 행동에 화가나 여러명이 사직한 기구의 이름은? | 역사교육과정개발추진위원회 | 역사교육과정개발추진위원회 | ✅ |
| mrc-1-000724 | 피어슨이 다시 의회를 해산했던 년도는? | 1965년 | 1965년 | ✅ |
| mrc-0-003727 | 이흥구의 사법시험 이야기를 기사로 작성한 곳은? | 〈중앙일보〉 | 〈중앙일보〉 | ✅ |
| mrc-0-003115 | 남북조 시대에서 이이 씨가 전쟁이 발생했을 때, 생활했던 장소는? | 미타케성 | 이이노야 성 | ❌ |
| mrc-0-003088 | 박지훈은 1라운드에서 몇 순위를 차지했는가? | 전체 4순위 | 전체 4순위 | ✅ |
| mrc-0-005296 | 데메카론에는 무엇을 풍자하는 이야기가 들어있나요? | 뇌물 | 뇌물 | ✅ |
| mrc-1-001495 | 병에 걸려 죽을 확률이 약 25~50%에 달하는 유형의 질병은? | 보통 유형 준융합성 천연두 | 보통 유형 준융합성 천연두 | ✅ |
| mrc-0-005289 | 설리반이 불만을 표시한 대상은 누구인가? | 데스탱 장군 | 데스탱 장군 | ✅ |
| mrc-0-002240 | 베소스는 어디서 추방당했는가? | 박트리아 | 박트리아 | ✅ |
| mrc-0-001846 | 자신의 이상적인 국가관이 스파르타와 닮아 있다고 생각하는 플라톤의 저서는? | 《국가》 | 《국가》 | ✅ |
| mrc-1-001653 | 박제된 북극곰이 사망한 날짜는? | 2011년 3월 19일 | 2011년 3월 19일 | ✅ |
| mrc-0-000785 | 문법 측면에서 더 보수적인 포르투갈어 표준은? | 유럽 포르투갈어 | 브라질 포르투갈어 | ❌ |
| mrc-0-001519 | 로스 수장이 살해한 사람은 어느 당 회원인가? | 메이저 릿지 | 릿지당 | ❌ |
| mrc-0-002457 | 조경숙왕의 아들인 요자의 친어머니는 누구인가? | 요희 | 강도역왕 | ❌ |
| mrc-1-001479 | 오래플린과 부스의 마지막 계획에 따르면 그들은 어디서 링컨을 납치하려고 했는가? | 포드 극장 | 포드 극장 | ✅ |
| mrc-0-001707 | 김득황이 친일파로 취급되었던 것은 무슨 경력 때문인가? | 만주국 관리 | 만주국 관리 | ✅ |
| mrc-0-002280 | 레닌이 출간한 책 중 농민의 자발적 참여에 대한 내용이 포함되어있는 것은? | 『협동조합에 관하여』 | 『협동조합에 관하여』 | ✅ |
| mrc-0-001807 | 류한욱이 두 번째 뇌출혈로 쓰러진 공간은? | 광주교도소 | 광주교도소 | ✅ |
| mrc-0-001161 | 로마의 공성무기에 대한 기록을 남긴 사람은? | 작가 베게티우스 | 베게티우스 | ❌ |
| mrc-0-004654 | 동반자 등록제를 최초로 실시한 중국의 도시는? | 가오슝 시 | 가오슝 시 | ✅ |
| mrc-0-002468 | 장대호가 사용한 흉기는? | 쇠망치 | 망치 | ❌ |
| mrc-0-001870 | 스뮈츠에게 학비를 지원해 준 사람은? | 마리즈 교수 | 마리즈 교수 | ✅ |
| mrc-1-000753 | 브루투스가 세운도시의 현재 이름은? | 런던 | 런던 | ✅ |
| mrc-0-001073 | 일본의 대학 입시는 며칠간 진행되는가? | 이틀 | 이틀 | ✅ |
| mrc-1-001116 | 국내 화엄종의 선구자는 누구인가? | 의상대사 | 의상대사 | ✅ |
| mrc-1-000995 | 다케다 노부히로가 통치한 지역은 어디인가? | 홋카이도 | 홋카이도로 | ❌ |
| mrc-0-002175 | 둥근 해자를 건너는 다리 난간에는 어떤 신화의 내용이 새겨져 있나? | ‘우유의 바다’ | ‘우유의 바다’ | ✅ |
| mrc-0-001894 | 우나동의 주요 식재료는? | 장어 | 장어 | ✅ |
| mrc-0-000337 | 제2차 세계 대전 이후 동부 갈리치아 지방은 누구에게 지배를 받았는가? | 소련 | 소련 | ✅ |
| mrc-0-002013 | 치환과 결합되어 파이스텔 암호 사용을 가능케 하는 것은? | 전치 | 전치 | ✅ |
| mrc-1-001766 | 적색육을 지칭하는 또 다른 이름은? | 흑색육(黑色肉) | 흑색육(黑色肉) | ✅ |
| mrc-0-001554 | 닛폰 제지 시라오이 공장과 가까운 역은? | 하기노 역 | 하기노 역 | ✅ |
| mrc-0-002247 | 벽에 천녀를 그리기 전에 하는 밑작업은? | 회칠 | 회칠 | ✅ |
| mrc-1-001177 | 독일의 취업자들이 주로 기술을 습득한 방법은? | 직업 교육 | 직업 교육 | ✅ |
| mrc-0-000939 | 김준연은 김구가 누구와 몰래 교류하고 있다고 주장했나요? | 공산당 | 공산당 | ✅ |
| mrc-0-000223 | 당산제를 시행하던 날은 언제였는가? | 매년 음력 정월 | 매년 음력 정월 | ✅ |
| mrc-0-005386 | 육조혜능의 이름을 모방한 불교의 종파는? | 조계종 | 조계종 | ✅ |
| mrc-0-000118 | 더우징퉁의 부모가 결혼한 곳은 어디인가? | 베이징 | 베이징 | ✅ |
| mrc-0-003529 | 벨기에가 리에주 전투에서 맞서 싸운 상대방은? | 독일군 | 독일군 | ✅ |
| mrc-1-000459 | 아킬레 라티가 추기경으로 추대된 해는? | 1932년 | 1932년 | ✅ |
| mrc-0-002931 | 서울 주변에서 자주 볼 수 없는 불상은? | 나말여초 | 나말여초 불상 | ❌ |
| mrc-0-002767 | 교황 비오 11세가 사망한 연도는? | 1939년 | 1939년 | ✅ |
| mrc-1-000796 | 엘디 서쪽 리틀강에 위차하는 마을은? | 도버 밀 | 도버 밀 | ✅ |
| mrc-0-003576 | 2012년 당시 곽태휘의 소속팀은? | 울산 | 울산 | ✅ |
| mrc-0-002692 | 교황의 문장에서 교차한 금빛 열쇠와 은빛 열쇠가 뜻하는 바는? | 땅(은색)과 하늘(금색)을 맺고 푸는 권한 | 땅(은색)과 하늘(금색)을 맺고 푸는 권한 | ✅ |
| mrc-0-004899 | 돌을 캐고 난 빈 채석장은 어떠한 용도로 이용하였나? | 저수지 | 저수지 | ✅ |
| mrc-1-000127 | 쿠데타를 위해 초대된 나폴레옹의 수하는? | 피에르 오주로 | 피에르 오주로 | ✅ |
| mrc-0-005412 | 엠넷미디어와 YG엔터테인먼트는 각자 상대방의 무엇을 얻어 협력관계를 맺었는가? | 지분 | 지분 10% | ❌ |
| mrc-0-004268 | 대한민국 민법에서 채권의 권리가 누구에게 있으면 채권이 사라지지 않는다고 명시하는가? | 제 3자 | 제 3자 | ✅ |
| mrc-1-000835 | 이베이의 계산 방법으로 아디옌이 많이 보급될수록 필요성이 없어지는 결제 방법은? | 페이팔 | 페이팔 | ✅ |
| mrc-1-001132 | 수를 놓을 때에 사용하는 기법 중 아주 작은 점으로 표현하는 방식은? | 점수 | 점수 | ✅ |
| mrc-0-002512 | 각룡류 중에서 가장 짧은 프릴을 가지고 있는 공룡은? | 코스모케라톱스 | 코스모케라톱스 | ✅ |
| mrc-0-003017 | 러시앤캐시는 광고를 통해 어떤 회사라는 이미지에서 탈피하고 싶어하는가? | '사채회사' | '사채회사' | ✅ |
| mrc-0-000215 | 1954년 샤넬이 프랑스로 돌아오기 전 머무르던 국가는? | 스위스 | 스위스 | ✅ |
| mrc-0-002873 | 경제학의 어떤 개념을 통해 개방경지제의 공동 방목지가 능률이 떨어지는 이유를 이해할 수 있는가? | "공유지의 비극" | "공유지의 비극" | ✅ |
| mrc-1-000384 | 와그너가 국내 건강보험과 민생을 위해 입법하는 것에 열정적으로 지지했던 인물은? | 해리 트루먼 대통령 | 해리 트루먼 대통령 | ✅ |
| mrc-0-005270 | 무위의 오랑캐들의 횡포로 길이 끊긴 사실을 소칙에게 알린 사람의 관직은? | 무위태수 | 무위태수 | ✅ |
| mrc-1-000158 | 충주 조동리유적의 존재 사실을 언제 알게 되었는가? | 1990년 9월 | 1990년 9월 | ✅ |
| mrc-0-002189 | 뱌쿠롄과 레이무 일행은 어떤 철학 때문에 싸우게 되었나? | "인간과 요괴의 완전한 평등" | "인간과 요괴의 완전한 평등" | ✅ |
| mrc-0-000905 | 견훤에게 살해당한 인물과 함께 태조를 대접한 왕은? | 경순왕 | 경순왕 | ✅ |
| mrc-0-001198 | 인도 총리에게 말라리아와 루푸스 치료제를 보내달라고 요구한 인물은? | 도널드 트럼프 미국 대통령 | 도널드 트럼프 미국 대통령 | ✅ |
| mrc-0-003947 | 1831년 조선 대목구의 첫 주교 직위를 맡은 사람은? | 바르톨로메오 브뤼기에르 | 바르톨로메오 브뤼기에르 신부 | ❌ |
| mrc-1-001328 | 탱화는 어느 종교와 관련이 있는가? | 불교 | 불교 | ✅ |
| mrc-0-000166 | 천체 관측과 동시에 고도와 방위각을 알 수 있는 망원경은? | 경위대식 망원경 | 경위대식 망원경 | ✅ |
| mrc-0-004090 | 철도보다 자가용 이용이 많아지면서, 히사쓰 오렌지 철도 노선의 대항마로 부상한 것은? | 남큐슈 자동차 전용 도로 | 남큐슈 자동차 전용 도로 | ✅ |
| mrc-0-003522 | 태양이 대부분 철로 이루어지지 않음을 발견한 인물은? | 세실리아 페인 | 세실리아 페인 | ✅ |
| mrc-1-001398 | 클라이언트이자 서버인 경우가 많아지는 상황에서 변화가 생기고 있는 환경은? | 웹 2.0 | 웹 2.0 | ✅ |
| mrc-0-000355 | 캐서린 스윈포드의 언니의 직업은? | 수녀 | 수녀 | ✅ |
| mrc-0-001590 | 시노하라에 의하면 아인슈타인은 무엇으로 원자폭탄 제작에 도움을 주었는가? | 상대성이론 | 상대성이론 | ✅ |
| mrc-1-001522 | 주이로부터 문흠의 가짜 항복 소식을 들은 인물은? | 손권 | 대제 손권 | ❌ |
| mrc-0-004307 | 콘스탄스를 죽인 것은 누구인가? | 350년 | 마그넨티우스 | ❌ |
| mrc-0-002471 | 댈러스 이전에 법무장관 직을 거부한 인물은 어디 출신인가요? | 펜실베이니아주 출신 | 펜실베이니아주 | ❌ |
| mrc-1-001313 | 로얄 살롱에 부착된 후면 번호판의 기존 위치는 어디인가? | 트렁크 | 트렁크에서 범퍼 | ❌ |
| mrc-0-004083 | 카르가 반대입장으로 돌아서자 반란세력이 진행한 것은? | 데코행진 | 데코행진 | ✅ |
| mrc-0-004197 | 어느정도 규모 이상의 대회에서 깃털 셔틀콕만을 사용하는 이유는? | 깃털 셔틀콕의 타구감을 선호하고, 또한 플라스틱보다 깃털 셔틀콕이 정교한 컨트롤을 하기에 보다 더 적합하기 때문이다 | 깃털 셔틀콕의 타구감을 선호하고, 또한 플라스틱보다 깃털 셔틀콕이 정교한 컨트롤을 하기에 보다 더 적합하기 때문 | ❌ |
| mrc-0-003208 | 한일생명 축구단의 감독에게 감사패를 준 축제가 열린 해는? | 1998년 | 1998년 | ✅ |
| mrc-1-000297 | 교황에 의해 새롭게 토지 개발된 최초의 관리지역은? | 드라 | 드라 | ✅ |
| mrc-1-000358 | 2003년 세계 태권도 선수권 대회가 개최되었던 장소는? | 가르미슈파르텐키르헨 | 가르미슈파르텐키르헨 | ✅ |
| mrc-1-001785 | 슘페터는 경제적 이윤도 전혀 발생시키지 않고 아무 발전이 없는 사회를 무엇이라 정의했나? | 정태적인 자본주의 | 정태적인 자본주의 | ✅ |
| mrc-0-005042 | 트레이 파커가 목소리를 맡은 등장인물의 자식은 누구인가? | 데미안 | 데미안 | ✅ |
| mrc-0-004677 | 받침 굽에 새겨져 있는 글자의 수는? | 34자 | 34자 | ✅ |
| mrc-0-003564 | 상사병으로 사망한 소녀의 장례식이 열린 곳은 어디인가? | 혼묘지 | 혼묘지(本妙寺) | ❌ |
| mrc-0-003753 | 생산물 중 다른 물건을 만들기 위해 사용되는 것은? | 중간 생산물 | 중간 생산물 | ✅ |
| mrc-0-005155 | 애니미즘만을 믿던 세부아노족들에게 전파된 외부 종교는? | 기독교 | 기독교 | ✅ |
| mrc-0-002679 | 제10군단의 아퀼리페르가 아퀼라를 든 채 배 밖으로 뛰어들었을 당시, 카이사르군대가 두려워한 대상은? | 브리튼인들 | 브리튼인 | ❌ |
| mrc-0-004527 | 《악마는 프라다를 입는다》은 어느 국가의 영화인가요? | 미국 | 미국 | ✅ |
| mrc-1-000037 | 마이어의 사인은? | 뇌졸중 | 뇌졸중 | ✅ |
| mrc-0-004092 | 장면을 가톨릭 성자로 추앙하려 했던 사람은? | 김수환 추기경 | 김수환 추기경 | ✅ |
| mrc-0-000707 | 석탄기 다음 시대는? | 페름기 | 펜실베이니아기 | ❌ |
| mrc-1-000658 | 돈대에서 무너져내린 포좌부분이 있는 위치는? | 서남쪽 | 서남쪽 | ✅ |
| mrc-0-000412 | 크레센티우스 2세가 받은 형벌은? | 성벽에 목이 매달리는 교수형 | 교수형 | ❌ |
| mrc-0-001288 | 왕필과 함께 대표적인 현학자로 불리며 《장자》에 주석을 단 사람은? | 곽상 | 곽상 | ✅ |
| mrc-1-000197 | 요시카와 하루오의 어린시절 역을 맡은 성우는? | 신도 케이 | 신도 케이 | ✅ |
| mrc-0-002253 | 반바리사이파의 반발을 불러일으킨 원인은 누구의 사망이었나요? | 디오게네스 | 디오게네스 | ✅ |
| mrc-1-000367 | 외국어영화상 위원회에서 최종 후보 다섯 편을 추리는 방법은? | 비밀 투표 | 비밀 투표 | ✅ |
| mrc-0-003906 | 쿠데타가 있었던 회차는? | 61화 | 61화 | ✅ |
| mrc-0-001254 | 스즈키 증장이 전사할 당시에 타고 있던 것은? | 카누 | 카누 | ✅ |
| mrc-0-004435 | 네이딘의 하나뿐인 친구는? | 크리스타 | 크리스타 | ✅ |
| mrc-0-000521 | 효율적인 운영을 도와주는 열차선 배치 방법은? | 속도별 배치 | 속도별 배치 | ✅ |
| mrc-1-000066 | 쓰치야 히로시가 마지막으로 활동한 해는? | 1975년 | 1975년 | ✅ |
| mrc-0-002462 | 예루살렘의 공방전이 지속된 기간은? | 석달 | 석달 | ✅ |
| mrc-0-002886 | 조선의 민족대표들이 조선이 독립국임을 선언한 곳은? | 태화관(서울시 종로구 인사동 소재) | 태화관 | ❌ |
| mrc-0-001646 | 1881년 리비에르의 목적지는? | 하노이 | 하노이 | ✅ |
| mrc-0-005046 | 성당의 서쪽과 남쪽 부분에 난방이 가능해진 시기는? | 1913년 | 1913년 | ✅ |
| mrc-1-001611 | 빔비사라가 왕위에 오른 뒤 마가다가 지배권을 얻은 도시는 어디인가? | 바라나시 | 라자그리하 | ❌ |
| mrc-0-005222 | 1835년 선출된 텍사스 지원군 지휘관이 제임스 보이에게 임명한 직책은? | 대령 | 대령 | ✅ |
| mrc-0-004879 | 2011년이후 세력이 약해진 조직은? | 걸프 카르텔(카르텔 델 골포: Cartel del golfo) | 걸프 카르텔 | ❌ |
| mrc-1-001481 | 확증편향이라는 단어를 처음으로 인용한 것은 누구인가? | 피터 케스카트 왓슨 | 피터 케스카트 왓슨 | ✅ |
| mrc-0-003637 | 입자들의 속도가 줄 때, 방출되는 에너지가 만들어내는 것은? | 충격파 | 충격파 | ✅ |
| mrc-0-003882 | 10월 1일 북진으로 압록강 근처까지 전쟁 지역을 이르게 한 부대는? | 대한민국 3보병사단 | 대한민국 3보병사단 | ✅ |
| mrc-1-001285 | 캠벨에게 원주민 성직자 양성의 중요성을 일깨워준 사람은? | 우리의 훌륭한 중국 동료 | 중국 동료 | ❌ |
| mrc-0-005109 | 학생들이 선생님과 같은 열정과 에너지를 가지게 되는 것은 무엇을 통해서인가? | 감정 전염 | 감정 전염 | ✅ |
| mrc-1-001455 | 토큰링이 빠른 속도로 퇴보하는데 영향을 준 기술은? | 스위치 이더넷 | 스위치 이더넷 | ✅ |
| mrc-0-001274 | 중고령자의 채취업 활성화로 경제 주체마저 옮겨가는 바람에 생기는 부작용 중 하나는? | 소비감소 | 소비감소 | ✅ |
| mrc-1-000855 | 1901년에 연기로 인해 타격을 받아 황폐한 마을이 되어버린 곳은? | 마츠키 촌 | 마츠키 촌 | ✅ |
| mrc-0-003587 | 조현병은 무엇으로 오해하기 쉬운가? | 사춘기에서 흔히 볼 수 있는 정서적 불안정성 | 정서적 불안정성 | ❌ |
| mrc-0-005031 | 탄수화물의 최종적 분해가 이루어지는 곳은? | 시트르산 회로 | 시트르산 회로 | ✅ |
| mrc-0-005154 | 수행에 대해 근본적으로 의문을 제기함으로써 영적 수행자의 수행 의지를 갉아먹는 것은 무엇인가? | 아케디아 | 아케디아 | ✅ |
| mrc-0-000477 | 알베르가 에마뉘엘을 "성 에마뉘엘"로 부른 것은 무엇 때문인가? | 에마뉘엘의 과시하는 듯한 신앙심 | 과시하는 듯한 신앙심 | ❌ |
| mrc-0-002513 | 네거티브 광고를 할 때 기업은 법적인 문제 외에 또 무엇을 생각해야 하는가? | 소비자들의 반응 | 소비자들의 반응 | ✅ |
| mrc-0-003022 | 아길루프의 요구에 의해 보니파시오 4세에게 서신을 쓴 사람은? | 아일랜드 선교자 골룸바노 | 골룸바노 | ❌ |
| mrc-1-000946 | 나오치카는 주살의 위험을 피해 누구의 영지로 도망갔나요? | 다케다 | 다케다 | ✅ |
| mrc-0-001110 | 《탄이초》의 작자에 가장 적합한 인물로 알려진 것은? | 유이엔 | 유이엔 | ✅ |
| mrc-0-003149 | 베르니츠 강과 라슈트 강이 만나는 곳 부근에서 코네티컷 연대와 만나 전투를 벌였던 프랑스 부대는 어디인가? | 트레뻬제 부대 | 트레뻬제 부대 | ✅ |
| mrc-0-004132 | 면역력이 약해져서 아시클로버의 약효가 발휘되지 않을 때 쓰는 것은 무엇인가? | 포스카르네트 | 포스카르네트 | ✅ |
| mrc-0-005300 | 동맹항에서의 교역에서 세금을 내지 않으려면 지참해야 하는 것은 무엇인가? | 면세증 | 면세증 | ✅ |
| mrc-0-004203 | 동궁전 화재에서 인종을 구한 사람은 누구인가? | 숙의 정씨 | 숙의 정씨 | ✅ |
| mrc-0-003566 | 굿보이'에 대한 비판글이 올라온 곳은? | 남성연대 홈페이지 | 남성연대 홈페이지 | ✅ |
| mrc-0-002926 | 봉은사 영산전 석가불상의 색을 다시 칠한 시기는? | 1895년 | 1895년 | ✅ |
| mrc-0-000536 | 코페르니쿠스의 지동설에 대한 내용으로 그의 사후에 발간된 책은? | 《천체의 회전에 관하여》 | 《천체의 회전에 관하여》 | ✅ |
| mrc-0-003848 | 요나미네가 야쿠자 팬으로부터 협박을 받았던 도시는 어디였는가? | 히로시마 | 히로시마 | ✅ |
| mrc-0-003760 | 박정희 전 대통령이 활주로가 있던 자리에 새로 지은 것은? | 5·16 광장(여의도 광장) | 5·16 광장(여의도 광장) | ✅ |
| mrc-0-001989 | 행정청사 옥상으로 피신했던 직원들이 알고 있었던 쓰나미의 높이는? | 6m | 6m | ✅ |
| mrc-0-004863 | 윤진학이 동아찬영회에 들어가기 전에 속해있었던 곳은? | 이등공송덕비건의소 | 이등공송덕비건의소 | ✅ |
| mrc-1-000714 | 태양풍을 멈추게 하는 것과 말단 충격의 틈을 무엇이라 부르는가? | 헬리오시스 | 헬리오시스 | ✅ |
| mrc-1-001173 | <석탄형성에 관한 관찰 기록>은 무엇에 관한 내용인가? | 탄광 | 탄광 환경 문제 | ❌ |
| mrc-0-005360 | 아이작 웨인의 아버지가 세상을 떠난 것은 몇 년도인가? | 1796년 | 1796년 | ✅ |
| mrc-1-000096 | 타코벨과 달리 대한민국에서 많은 성과를 달성했던 업체는? | 피자헛 | 피자헛 | ✅ |
| mrc-0-003467 | 8월 18일 대구역 근처에 박격포를 쏜 적군은 어디에서 침투했나? | 가산 | 가산 | ✅ |
| mrc-1-000593 | 1988년에 영국에 있는 파커의 공장을 찾아온 정치인은? | 마거릿 대처 전 영국수상 | 마거릿 대처 전 영국수상 | ✅ |
| mrc-1-000132 | 브라니미르의 뒤를 이어 크로아티아 공작의 직위를 승계받은 인물은? | 트르피미로비치 왕조(Trpimirović) 출신의 문치미르 | 문치미르 | ❌ |
| mrc-0-000787 | 볼드윈이 "당신들은 나를 야유합니까?"라는 말을 한 연도는? | 1947년 | 1947년 | ✅ |
| mrc-0-001494 | 대형 변압기 수송에 사용되는 전용선은? | 아이치 전기 전용선 | 아이치 전기 전용선 | ✅ |
| mrc-0-004411 | 자기장의 강도가 높아짐에 따라 페르미 면 외부로 옮겨지는 것의 명칭은? | 란다우 튜브 | 란다우 튜브 | ✅ |
| mrc-1-001622 | 나가시마의 은퇴경기는 무엇 때문에 미뤄졌나? | 우천 | 우천 | ✅ |
| mrc-0-002887 | 그녀의 첫 작품이 데뷔한 곳은? | ＜젊은 근위대＞ | ＜젊은 근위대＞ | ✅ |
| mrc-0-003437 | 송이 금에게 바친 공물은 은 30만량과 비단 몇 필인가? | 10만필 | 10만필 | ✅ |
| mrc-0-005286 | 카를로 말라테스타가 포로로 잡혀간 전쟁은? | 차고나라 전투 | 차고나라 전투 | ✅ |
| mrc-1-000295 | 푸에르토리코의 미카엘 곤살레스 선수가 참가하고자 했던 동계올림픽의 종목은? | 봅슬레이 | 봅슬레이 | ✅ |
| mrc-0-001253 | 미국 남부 지방에서 급격하게 영국산 제품이 많이 팔린 계기가 된 조치는? | 수입 금지 조치 | 수입 금지 조치 | ✅ |
| mrc-0-005324 | 악단이 재정적으로 힘들 때 단원들은 어디서 돈을 벌었는가? | 카바레나 클럽 | 카바레나 클럽 | ✅ |
| mrc-1-000983 | 블랙워치의 지도자가 게임에서 불리는 이름은 무엇인가? | 리퍼 | 리퍼와 솔저: 76 | ❌ |
| mrc-0-002718 | 통합진보당 해산 청구에 관한 안건을 의결한 기구는? | 국무회의 | 국무회의 | ✅ |
| mrc-0-003850 | 배현경과 신숭겸 등이 일을 꾀하여 찾아간 인물의 부인은 누구인가? | 제1부인 | 태조 왕건 | ❌ |
| mrc-0-001650 | 한국을 '형제의 나라'라고 칭한 나라는 어디인가요? | 멕시코 | 멕시코 | ✅ |
| mrc-0-004662 | 정상적인 조건에서 아라키돈 산이 염증을 일으키는데 필요한 물질은? | 지질 과산화 생성물 | 지질 과산화 생성물 | ✅ |
| mrc-0-003759 | 페루치가 피사에서 한달을 지체한 이유는 무엇인가? | 독감 | 독감 | ✅ |
| mrc-1-001279 | 갑훈에 의해 1천만여 전의 뇌물을 받았다는 사실이 밝혀진 인물은? | 양당(楊黨) | 양당(楊黨) | ✅ |
| mrc-0-001960 | 성직자들의 실체에 대해 말한 용병의 기사는 누가 작성하였나요? | 슈바이츠 암 존탁 | 슈바이츠 암 존탁 | ✅ |
| mrc-0-001162 | 러셀은 누구의 가족과 함께 1889년 파리 박람회에 갔는가? | 퀘이커 교도였던 앨리스 페어살 스미스 | 앨리스 페어살 스미스 | ❌ |
| mrc-0-004565 | 1표로 간주하기 위한 의견 수렴 방식은? | 다수결 | 다수결 득표 | ❌ |
| mrc-0-000754 | 데이비드 스튜어트는 누구의 해독방법을 다른 지협 문자 자료에 적용해보았나요? | 코프먼과 저스티슨 | 코프먼과 저스티슨 | ✅ |
| mrc-1-000024 | 인적 성과와 더불어 기업의 성과를 평가하는 요소는? | 물적 성과(物的成果) | 물적 성과 | ❌ |
| mrc-0-002095 | 공놀이 경기장 중 일부는 어디에 위치하고 있나? | '일곱 개의 신전 광장' | '일곱 개의 신전 광장' | ✅ |
| mrc-0-003083 | 창씨개명령의 시행일을 미루는 것을 수락한 인물은? | 미나미 지로 | 미나미 지로 총독 | ❌ |
| mrc-0-002978 | 망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가? | 200,000명 | 200,000명 | ✅ |

## 베이스 모델이 맞추고 어쩌구 저쩌구 모델이 틀린 항목들

| ID | 질문 | 정답 | 베이스 예측 | 어쩌구 저쩌구 모델 예측 |
| --- | --- | --- | --- | --- |
| mrc-0-000248 | 진전사의 명칭이 드러나는 데 영향을 준 물건은? | '진전(陳田)'이라 새겨진 기와조각 | '진전(陳田)'이라 새겨진 기와조각 | 기와조각 |
| mrc-0-002333 | 정민과 이별한 이후 옥림의 매력에 마음을 빼앗겨버린 인물은? | 욱 | 욱 | 아인 |
| mrc-1-001814 | 슈트레제만이 이끌었던 당은 무엇인가? | 독일인민당 | 독일인민당 | 독일 인민당 |
| mrc-0-001311 | 진행 천궁도의 가치는 무엇의 형태에 영향을 받나요? | 출생 천궁도 | 출생 천궁도 | 출생 천궁 |
| mrc-1-000961 | 닭의 관절 재생을 유도할 수 있는 재생 기술은? | 창 절제술 | 창 절제술 | 창 절제술이나 절편 절제술 |
| mrc-0-003118 | 쿠에바가 생각하는 자본주의 체제는? | 그 자체가 계급 모순을 가지는 체제 | 그 자체가 계급 모순을 가지는 체제 | 계급 모순을 가지는 체제 |
| mrc-0-005186 | 팔라바국이 9세기 초까지 텔랑가나와 타밀나두 북쪽 지역을 다스린 기간은? | 약 600년 | 약 600년 | 600년 |
| mrc-1-000516 | 아이오와주에 수많은 산업체들이 들어오기 시작한 것은 언제인가? | 1945년 | 1945년 | 1945년과 1960년대 후반 사이 |
| mrc-0-001980 | 161개의 집터 근처에서 발견된 석실묘의 개수는? | 1기 | 1기 | 171기 |
| mrc-1-001446 | 기렌 총수가 살해당한 직후에 파괴된 군함은? | 대형 항공 모함 도로스 | 대형 항공 모함 도로스 | 화이트 베이스 |
| mrc-0-001952 | 어피인장함의 뒷면에 붙은 경첩은 무엇의 형태인가? | 제비초리 | 제비초리 | 제비초리형태 |
| mrc-0-003211 | 여자가 안드레우치오의 디 피에트로에 대한 정보를 얻었던 인물은? | 할머니 | 할머니 | 한 할머니 |
| mrc-0-000159 | 마오를 직접적으로 죽인 사람은? | 일등병 토머스 | 일등병 토머스 | 토머스 |
| mrc-0-004725 | 워너가 자신의 회사를 매도했던 해는? | 1966년 | 1966년 | 1956년 |
| mrc-1-001270 | 샤이닝 폼을 무엇이라고 칭하기도 하나요? | "광휘에의 각성"(코키에노 메자메) | "광휘에의 각성"(코키에노 메자메) | 광휘에의 각성"(코키에노 메자메) |
| mrc-0-001552 | 화순 춘산영당에 소장된 최익현의 초상화는 무엇에다 그린 것인가? | 비단 | 비단 | 석지 채용신 |
| mrc-0-003262 | 십자군 설교를 통해 집한한 자들은? | 빈자의 십자군 | 빈자의 십자군 | 맘루크를 몰아내기 위한 십자군 설교를 지시했다. 그 결과 1309년 7월 아비뇽 성문 앞에 일명 빈자의 십자군 |

## 어쩌구 저쩌구 모델이 맞추고 베이스 모델이 틀린 항목들

| ID | 질문 | 정답 | 어쩌구 저쩌구 모델 예측 | 베이스 예측 |
| --- | --- | --- | --- | --- |
| mrc-1-000414 | 다른 과 의사들은 감염내과 전문의들로부터 어떤 것에 대해 조언을 받는가? | 복잡한 감염병 | 복잡한 감염병 | 감염병 환자를 진단하거나 관리하기 어려운 경우 |
| mrc-0-003032 | 반대동맹이 공산당과 갈라서겠다고 얘기한 날은 언제인가? | 1967년 11월 15일 | 1967년 11월 15일 | 1967년 8월 16일 |
| mrc-0-001022 | 칭자오의 머리가 엄청난 위력을 발휘할 수 없게 된 것은 누구 때문인가? | 몽키 D. 가프 | 몽키 D. 가프 | 루피의 할아버지 |
| mrc-0-001150 | 해초나 조류 표면에서 자라는 유기체 중 가장 비율이 높은 것은? | 조류 | 조류 | 광합성을 하는 착생식물 |
| mrc-0-000547 | 합천에서 나루터 역할을 대신하고 있는 것은? | 황강다리 | 황강다리 | 성산리의 느티나무 |
| mrc-0-003801 | 제나라가 쇠약해진 시기는? | 제서지전(齊西之戰)이후 | 제서지전(齊西之戰)이후 | 제서지전(齊西之戰) |
| mrc-1-000449 | 몽양 여운형의 아버지 이름은 무엇인가요? | 여정현(呂正鉉) | 여정현(呂正鉉) | 경주 이씨 |
| mrc-0-004133 | 부처의 몸에서 뿜어져 나오는 빛을 형상화한 것은 무엇인가? | 광배 | 광배 | 배(舟) |
| mrc-0-001704 | 일본 프로 야구에서 처음으로 가쿠사다마를 사용한 선수는? | 가리타 히사노리 | 가리타 히사노리 | 가리타 히사노리에 |
| mrc-0-002981 | 요미우리가 먼저 에가와를 지명할 수 없게 만든 구단은? | 크라운라이터 | 크라운라이터 | 크라운라이터 라이온스 |
| mrc-0-002906 | 메테스키가 최초로 폭탄을 설치하였던 지역은? | 맨해튼 | 맨해튼 | 맨해튼에 위치한 컨솔러데이티드 에디슨 발전소 창턱 |
| mrc-0-004202 | 러시아에 있는 다량의 천연자원을 약탈하려했던 계획은? | 바르바로사 작전 | 바르바로사 작전 | 바르바로사 |
| mrc-0-004015 | 김준연은 김옥성과 몇 년간 함께 살았나요? | 60년간 | 60년간 | 60년 |
| mrc-0-003289 | 전자부품에 특화된 종합물류서비스를 제공하는 기업의 모회사는? | 알프스 전기 주식회사 | 알프스 전기 주식회사 | 일본의 다국적기업 알프스 전기 주식회사 |
| mrc-1-000918 | 비슷한 양의 골편이 두종류가 발견되는것으로 알 수 있는 A.수페르스테스의 몸의 형태는? | 좌우대칭 | 좌우대칭 | 낫\n형 |
| mrc-0-001240 | 기타 케이스에 기타 외의 물건이 있는 인물은 어떤 역할 인가요? | 악당 | 악당 | 엘마리아치 |
| mrc-0-001001 | 소프트웨어 공학이 이전과는 다른 분야로 나아가게 된 계기는? | 디지털 컴퓨터 | 디지털 컴퓨터 | 디지털 컴퓨터가 등장 |
| mrc-1-001121 | 미래의 군사 능력은 무엇의 능력에 의해 결정되는가? | 군대 | 군대 | 가능한 다양한 미래 시나리오의 도전을 해결할 수있는 군대 |
| mrc-1-000507 | 동도서기라는 구호를 사용한 나라에서 처음 서구화와 접한 시기는? | 17세기 조선 시대 | 17세기 조선 시대 | 17세기 |
| mrc-0-005105 | 정부의 성향이 진보에 가까울 때 사법적극주의를 띠는 법원은 어떤 성향을 갖나? | 보수 | 보수 | 보수 성향 |
| mrc-0-003057 | 센니치마에 선의 관리권은 어디에 있나? | 오사카 | 오사카 | 돔마에 역 |
| mrc-0-000484 | 전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가? | 제 양왕 | 제 양왕 | 기겁 |

## 둘 다 틀린 항목들

| ID | 질문 | 정답 | 어쩌구 저쩌구 모델 예측 | 베이스 예측 |
| --- | --- | --- | --- | --- |
| mrc-0-002138 | 몽케가 죽은 뒤 쿠릴타이에서 대칸의 지위를 얻은 사람의 이름은? | 아리크 부케 | 쿠빌라이 | 막냇동생 아리크 부케 |
| mrc-1-001023 | 신란의 동반자가 죽었다고 전해지는 지역은? | 도고쿠 | 사이넨지(西念寺) | 사이넨지 |
| mrc-0-004342 | 갈라파고스 제도의 9월 날씨에 무엇이 계속 오는가? | 가루아 | 이슬비 | 잦은 이슬비(가루아) |
| mrc-0-003677 | 러셀의 여자 친구의 종교는? | 퀘이커 교 | 퀘이커 | 퀘이커 교도 |
| mrc-0-000561 | 할더가 히틀러의 공격안을 반대한 결과는? | 독일 | 독일군의 대참패 | 스탈린그라드 전투에서 독일군의 대참패 |
| mrc-1-000839 | 뒤러는 어디에 코뿔소를 그렸는가? | 목판 | 딱딱한 판으로 몸이 둘러싸인 동물 | 목 |
| mrc-0-001486 | 냉대 동계 소우 기후의 w는 무엇의 약자인가? | 동계건조(wintertrocken) | 동계건조 | 동계건조(wintertrocken)을 가리킨다. 트란스 바이칼 기후 |
| mrc-1-000291 | 이인국 박사가 석방되는 결과를 가져온 병은? | 혹 | 이질 | 스텐코프의 혹 |
| mrc-0-003033 | 동기부여적 설명이 효과가 있는지 알아내기 위해 현재 연구되고 있는 것은 무엇인가? | 귀인 평향과 행동의 상관 관계 | 특정한 귀인 평향과 행동의 상관 관계 | 현재의 연구 |
| mrc-0-005478 | 코로나를 발생시키는 것은? | 높은 전위의 전극 | 전류 | 공기와 같은 중성유체 속의 높은 전위의 전극 |
| mrc-0-004495 | 송나라가 돈을 지불하여 피하고자 했던 것은? | 전쟁 | 평화 | 전쟁을 일으키는 것보다 차라리 어느 정도의 돈을 내고 평화 |
| mrc-0-003844 | 바이마르 정부가 화폐를 수없이 찍어낸 이유는? | 배상금을 주기 위하여 | 배상금 | 배상금을 주기 위하 |
| mrc-0-005355 | 본국 의회가 제정한 선언법은 식민지와 관련하여 무엇을 의미하는가? | 본국 의회가 식민지에 대한 입법을 “어떤 경우에도 행할 수” 있도록 하였다 | 주권 | 본국 의회가 식민지에 대한 입법을 “어떤 경우에도 행할 수” |
| mrc-1-000899 | 길버트 J. 체크 중령이 7월 23일 저녁에 공격을 막기 위해 준비한 곳은 어디 주변이었나? | 보은군 남쪽 상용리(현 영동군 용산면 상용리) 마을 | 상용리 | 상용리(현 영동군 용산면 상용리) 마을 근처 |
| mrc-0-004454 | Mr.9은 라분을 무엇으로 사용하려고 했는가? | 식재료 | 현상금 사냥꾼들의 식재료 | 같은 회사의 현상금 사냥꾼들의 식재료를 구하기 위해 |
| mrc-1-001534 | 거북이가 더 나쁘다고 하는 사람들이 문제삼는 것은? | 공정 | 거북이가 토끼를 보고도 그냥 뛰어갔기에 공정 | 거북이가 토끼를 보고도 그냥 뛰어갔기에 |
| mrc-1-001567 | 아카키우스 분열은 얼마나 유지됐는가? | 35년 | 35년간 | 519년까지 |
| mrc-0-003146 | 타지마할의 실제 구조를 밝히는데 도움을 준 것은? | '달빛 정원'(Moonlight Garden) | '달빛 정원' | 야무나 강 반대쪽에 '달빛 정원'(Moonlight Garden) |
| mrc-0-004837 | 조르댕의 딸은 누구를 좋아하는가? | 클레몽트 | 도랑트 백작 | 루씰은 |
| mrc-0-001719 | 극처럼 점단위 공격이 가능한 무기는? | 모 | 방천화극 | 장극 |
| mrc-0-003752 | 제6회 전조선축구대회가 진행된 기간은? | 사흘 | 사흘간 | 1925년 4월 28일부터 사흘 |
| mrc-0-004530 | 대제학을 지낼 수 있는 유일한 사람은? | 모든 관직을 거친 대과 급제자 | 대과급제자 | 대과 급제자 |
| mrc-1-000622 | 마르크스주의자들의 사상은? | 공산주의 | 비밀결사주의 | 기존 마르크스주의자들 |
